# การ Fine-tuning ด้วย QLoRA โดยใช้ Unsloth

<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a>
</div>

## วัตถุประสงค์การเรียนรู้

ใน lab นี้ จะได้:
1. เข้าใจว่า QLoRA (Quantized Low-Rank Adaptation) คืออะไรและทำงานอย่างไร
2. เตรียมและจัดรูปแบบชุดข้อมูลสำหรับการ fine-tuning แบบ instruction
3. กำหนดค่าพารามิเตอร์ LoRA (rank, alpha, dropout, target modules)
4. Fine-tune โมเดล Gemma 2 2B บนชุดข้อมูล instruction-following
5. เปรียบเทียบประสิทธิภาพระหว่างโมเดลพื้นฐานกับโมเดลที่ fine-tune แล้ว
6. วิเคราะห์ประสิทธิภาพด้านหน่วยความจำของ QLoRA เทียบกับการ fine-tuning แบบเต็ม
7. บันทึกและส่งออกโมเดลที่ fine-tune แล้วเพื่อการใช้งาน

## QLoRA คืออะไร?

**QLoRA (Quantized Low-Rank Adaptation)** เป็นการรวมเทคนิค:

1. **Quantization**: ลดน้ำหนักของโมเดลเหลือ 4-bit precision เพื่อประหยัดหน่วยความจำ
2. **LoRA**: ฝึก adapter layers ขนาดเล็กแทนการฝึกทั้งโมเดล

### ทำไมต้องใช้ QLoRA?

- **ประหยัดหน่วยความจำ**: Fine-tune โมเดลขนาดใหญ่บน GPU สำหรับผู้บริโภคทั่วไป
- **การ training ที่รวดเร็ว**: อัพเดทเพียง 1-2% ของพารามิเตอร์โมเดล
- **คุณภาพสูง**: ได้ประสิทธิภาพใกล้เคียงกับการ fine-tuning แบบเต็ม
- **พกพาได้**: LoRA adapters มีขนาดเล็ก (MBs แทนที่จะเป็น GBs) และสามารถแชร์ได้

### LoRA ทำงานอย่างไร

แทนที่จะอัพเดทน้ำหนักทั้งหมด `W`, LoRA จะเพิ่มเมทริกซ์ที่ฝึกได้:
```
W_new = W_frozen + (LoRA_A × LoRA_B)
```
โดยที่:
- `W_frozen`: น้ำหนักต้นฉบับที่ถูกตรึง (quantized เป็น 4-bit)
- `LoRA_A, LoRA_B`: เมทริกซ์ขนาดเล็กที่ฝึกได้ (rank r << hidden_dim)

สำหรับการรัน lab นี้ ให้กด "*Runtime*" และกด "*Run all*" บน **free** Tesla T4 Google Colab instance!

## ส่วนที่ 1: การติดตั้ง

ก่อนอื่น เรามาติดตั้ง libraries ที่จำเป็น เราจะติดตั้ง Unsloth ซึ่งให้การสนับสนุน QLoRA

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

## ส่วนที่ 2: โหลดโมเดลพื้นฐานด้วย 4-bit Quantization

เราจะโหลด Gemma 2 2B ใน 4-bit precision นี่คือรากฐานของ QLoRA - โมเดลจะถูกตรึงไว้ที่ 4-bit ในขณะที่เราฝึก LoRA adapters

In [ ]:
from unsloth import FastModel
import torch

# Clear GPU memory
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

print("\n" + "="*60)
print("LOADING BASE MODEL WITH 4-BIT QUANTIZATION")
print("="*60)

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-it",
    max_seq_length = 2048,
    load_in_4bit = True,  # QLoRA uses 4-bit quantization
    dtype = None,  # Auto-detect dtype
)

# Track initial memory
base_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"\nBase model loaded: {base_memory} GB")
print(f"Model dtype: {model.dtype}")

## ส่วนที่ 3: กำหนดค่า LoRA Adapters

ตอนนี้เราจะเพิ่ม LoRA adapters ให้กับโมเดล สิ่งเหล่านี้คือพารามิเตอร์เพียงอย่างเดียวที่จะถูกฝึก

### พารามิเตอร์ LoRA หลัก:

- **r (rank)**: มิติของเมทริกซ์ LoRA ยิ่งสูง = ความจุมากขึ้นแต่ใช้หน่วยความจำมากขึ้น
  - ค่าทั่วไป: 8, 16, 32, 64
  - เราจะใช้ 16 เพื่อความสมดุลที่ดี

- **lora_alpha**: ตัวคูณสำหรับการอัพเดท LoRA มักจะตั้งเป็น `r` หรือ `2*r`
  - ควบคุมขนาดของการอัพเดท adapter
  - เราจะใช้ 16 (เท่ากับ r)

- **lora_dropout**: ความน่าจะเป็นของ dropout สำหรับเลเยอร์ LoRA
  - ป้องกัน overfitting บนชุดข้อมูลขนาดเล็ก
  - เราจะใช้ 0 (Unsloth แนะนำค่านี้)

- **target_modules**: เลเยอร์ไหนที่จะเพิ่ม LoRA adapters
  - ทั่วไป: ["q_proj", "k_proj", "v_proj", "o_proj"] สำหรับ attention
  - สามารถรวม ["gate_proj", "up_proj", "down_proj"] สำหรับ FFN

In [ ]:
print("\n" + "="*60)
print("ADDING LoRA ADAPTERS")
print("="*60)

model = FastModel.get_peft_model(
    model,
    r = 16,  # LoRA rank
    lora_alpha = 16,  # LoRA scaling
    lora_dropout = 0,  # No dropout (Unsloth optimized)
    
    # Target modules: attention layers
    target_modules = [
        "q_proj",  # Query projection
        "k_proj",  # Key projection
        "v_proj",  # Value projection
        "o_proj",  # Output projection
        "gate_proj",  # FFN gate
        "up_proj",  # FFN up
        "down_proj",  # FFN down
    ],
    
    bias = "none",  # Don't train bias terms
    use_gradient_checkpointing = "unsloth",  # Memory efficient
    random_state = 3407,
)

# Check trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
trainable_percent = 100 * trainable_params / all_params

print(f"\nTrainable parameters: {trainable_params:,}")
print(f"Total parameters: {all_params:,}")
print(f"Percentage trainable: {trainable_percent:.2f}%")

# Memory after adding LoRA
lora_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"\nMemory with LoRA adapters: {lora_memory} GB")
print(f"Additional memory for LoRA: {lora_memory - base_memory:.3f} GB")

## ส่วนที่ 4: เตรียมชุดข้อมูล

เราจะใช้ชุดข้อมูล **Alpaca** - คอลเลกชันของตัวอย่าง instruction-following 52K รายการ

### รูปแบบชุดข้อมูล

แต่ละตัวอย่างมี:
- **instruction**: คำอธิบายงาน
- **input**: บริบทหรืออินพุตเสริม (อาจจะว่าง)
- **output**: ผลลัพธ์ที่ต้องการ

เราจะจัดรูปแบบนี้ให้เป็น chat template ของ Gemma

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

# Load dataset
print("\n" + "="*60)
print("LOADING ALPACA DATASET")
print("="*60)

dataset = load_dataset("yahma/alpaca-cleaned", split="train")

print(f"\nDataset size: {len(dataset):,} examples")
print("\nExample from dataset:")
print("="*60)
print(f"Instruction: {dataset[0]['instruction']}")
print(f"Input: {dataset[0]['input']}")
print(f"Output: {dataset[0]['output']}")
print("="*60)

### ตั้งค่า Chat Template

เราจะกำหนดค่า chat template ของ Gemma เพื่อจัดรูปแบบข้อมูลของเราอย่างถูกต้อง

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma",  # Use Gemma format
)

print("Chat template configured!")

### จัดรูปแบบชุดข้อมูล

แปลงรูปแบบ Alpaca เป็นรูปแบบ chat ของ Gemma

In [ ]:
def format_alpaca_to_chat(example):
    """Convert Alpaca format to chat format"""
    
    # Combine instruction and input
    if example['input'].strip():
        user_message = f"{example['instruction']}\n\n{example['input']}"
    else:
        user_message = example['instruction']
    
    # Create chat messages
    messages = [
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": example['output']},
    ]
    
    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = False,
    )
    
    return {"text": text}

# Apply formatting
print("\nFormatting dataset...")
dataset = dataset.map(format_alpaca_to_chat)

# Show formatted example
print("\nFormatted example:")
print("="*60)
print(dataset[0]['text'][:500] + "...")
print("="*60)

### สร้างชุดข้อมูล Train/Test

เราจะใช้ 1000 ตัวอย่างสำหรับการ training (เพื่อให้การ training รวดเร็ว) และ 100 ตัวอย่างสำหรับการทดสอบ

In [ ]:
# For this lab, we'll use a subset for faster training
train_dataset = dataset.select(range(1000))  # First 1000 examples
test_dataset = dataset.select(range(1000, 1100))  # Next 100 examples

print(f"Training examples: {len(train_dataset):,}")
print(f"Test examples: {len(test_dataset):,}")

## ส่วนที่ 5: ทดสอบโมเดลพื้นฐาน (ก่อนการ Fine-tuning)

มาทดสอบโมเดลพื้นฐานด้วย prompts สองสามตัวเพื่อสร้างเกณฑ์อ้างอิง

In [ ]:
def test_model(model, tokenizer, prompt, max_tokens=200):
    """Helper function to test model with a prompt"""
    messages = [{"role": "user", "content": prompt}]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        tokenize = True,
        return_tensors = "pt",
        return_dict = True,
    ).to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens = max_tokens,
        temperature = 0.7,
        top_p = 0.9,
        do_sample = True,
    )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response

# Test prompts
test_prompts = [
    "Write a haiku about artificial intelligence.",
    "Explain the concept of recursion in programming in simple terms.",
    "What are three tips for staying productive while working from home?",
]

print("\n" + "="*60)
print("BASE MODEL RESPONSES (BEFORE FINE-TUNING)")
print("="*60)

base_responses = []
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n[Test {i}] Prompt: {prompt}")
    print("-"*60)
    response = test_model(model, tokenizer, prompt)
    base_responses.append(response)
    print(f"Response: {response}")
    print("-"*60)

## ส่วนที่ 6: กำหนดค่าการ Training

ตอนนี้เราจะตั้งค่าการ training โดยใช้ Library TRL

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

print("\n" + "="*60)
print("CONFIGURING TRAINING")
print("="*60)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    
    args = TrainingArguments(
        output_dir = "outputs",
        num_train_epochs = 1,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        learning_rate = 2e-4,
        warmup_steps = 5,
        optim = "adamw_8bit",
        logging_steps = 10,
        eval_strategy = "steps",
        eval_steps = 50,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        save_strategy = "epoch",
        save_total_limit = 1,
        seed = 3407,
        report_to = "none",  # Disable wandb, tensorboard, and all logging
    ),
)

print("\nTraining configured! Ready to train.")
print(f"  - Training examples: {len(train_dataset):,}")
print(f"  - Effective batch size: 8")

## ส่วนที่ 7: การ train โมเดล

In [ ]:
import time

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

torch.cuda.reset_peak_memory_stats()
start_time = time.time()

# Train!
trainer_stats = trainer.train()

training_time = time.time() - start_time
peak_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)
print(f"Training time: {training_time/60:.2f} minutes")
print(f"Peak GPU memory: {peak_memory} GB")
print(f"Final loss: {trainer_stats.training_loss:.4f}")
print("="*60)

## ส่วนที่ 8: ทดสอบโมเดลที่ Fine-tune แล้ว

มาทดสอบโมเดลที่ fine-tune แล้วด้วย prompts เดิมและเปรียบเทียบกัน

In [ ]:
print("\n" + "="*60)
print("FINE-TUNED MODEL RESPONSES (AFTER FINE-TUNING)")
print("="*60)

finetuned_responses = []
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n[Test {i}] Prompt: {prompt}")
    print("-"*60)
    response = test_model(model, tokenizer, prompt)
    finetuned_responses.append(response)
    print(f"Response: {response}")
    print("-"*60)

### การเปรียบเทียบแบบเคียงข้าง

In [ ]:
print("\n" + "="*80)
print("BEFORE vs AFTER COMPARISON")
print("="*80)

for i, prompt in enumerate(test_prompts):
    print(f"\n[Test {i+1}] {prompt}")
    print("-"*80)
    print(f"BEFORE: {base_responses[i]}")
    print(f"\nAFTER:  {finetuned_responses[i]}")
    print("="*80)

## ส่วนที่ 9: การวิเคราะห์การใช้หน่วยความจำ

มาวิเคราะห์ว่า QLoRA ประหยัดหน่วยความจำได้มากแค่ไหนเมื่อเทียบกับการ fine-tuning แบบเต็ม

In [ ]:
import matplotlib.pyplot as plt

methods = ['QLoRA\n(This Lab)', 'Full Fine-tuning\n(BF16)', 'Full Model\n(BF16)']
memory_usage = [peak_memory, 16, 4]
colors = ['#2ecc71', '#e74c3c', '#f39c12']

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(methods, memory_usage, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

for bar, mem in zip(bars, memory_usage):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{mem:.1f} GB',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('GPU Memory Usage (GB)', fontsize=12, fontweight='bold')
ax.set_title('Memory Efficiency: QLoRA vs Full Fine-tuning\nGemma-2 2B Model', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3, linestyle='--')

savings = ((16 - peak_memory) / 16) * 100
textstr = f'QLoRA Memory Savings:\n~{savings:.0f}% vs Full Fine-tuning'
props = dict(boxstyle='round', facecolor='lightgreen', alpha=0.8)
ax.text(0.98, 0.97, textstr, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', horizontalalignment='right', bbox=props)

plt.tight_layout()
plt.show()

print(f"\nQLoRA used only ~{(peak_memory/16)*100:.0f}% of the memory required for full fine-tuning!")